In [7]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = None
secret_key = None

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [8]:
from dotenv import load_dotenv

load_dotenv()

import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [9]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    %pip install alpaca-py
    import alpaca

In [10]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)
from alpaca.trading.requests import (
    ClosePositionRequest,
    GetAssetsRequest,
    GetOrdersRequest,
    LimitOrderRequest,
    MarketOrderRequest,
    StopLimitOrderRequest,
    StopLossRequest,
    StopOrderRequest,
    TakeProfitRequest,
    TrailingStopOrderRequest,
)
from alpaca.trading.enums import (
    AssetExchange,
    AssetStatus,
    OrderClass,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)

In [11]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [12]:
# check version of alpaca-py
alpaca.__version__

'0.42.0'

# Getting Data


In [13]:
# setup stock historical data client
stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [14]:
symbol = "AAPL"

In [18]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    # limit = 2,                                               # specify limit
)
stock_historical_data_client.get_stock_bars(req).df

open    high     low   close     volume  \
symbol timestamp                                                               
AAPL   2025-07-21 08:00:00+00:00  211.220  211.88  211.22  211.77    29973.0   
       2025-07-21 09:00:00+00:00  211.780  211.95  211.78  211.90    16739.0   
       2025-07-21 10:00:00+00:00  211.940  212.70  211.91  212.40    70925.0   
       2025-07-21 11:00:00+00:00  212.280  212.65  212.07  212.55    58897.0   
       2025-07-21 12:00:00+00:00  212.310  213.00  211.61  212.44  1811300.0   
...                                   ...     ...     ...     ...        ...   
       2025-07-25 19:00:00+00:00  213.790  214.00  213.40  213.96  5992074.0   
       2025-07-25 20:00:00+00:00  213.960  214.01  213.70  214.00  5289653.0   
       2025-07-25 21:00:00+00:00  214.020  214.03  213.81  213.90  1473927.0   
       2025-07-25 22:00:00+00:00  213.881  214.00  213.85  213.92    31347.0   
       2025-07-25 23:00:00+00:00  213.950  214.05  213.91  214.00    21447.0   

                                  trade_count        vwap  
symbol timestamp                                           
AAPL   2025-07-21 08:00:00+00:00       1315.0  211.692770  
       2025-07-21 09:00:00+00:00        676.0  211.881146  
       2025-07-21 10:00:00+00:00       1695.0  212.316094  
       2025-07-21 11:00:00+00:00       1870.0  212.456798  
       2025-07-21 12:00:00+00:00       7692.0  212.466799  
...                                       ...         ...  
       2025-07-25 19:00:00+00:00      78114.0  213.784071  
       2025-07-25 20:00:00+00:00       2031.0  213.879598  
       2025-07-25 21:00:00+00:00        507.0  213.954315  
       2025-07-25 22:00:00+00:00        348.0  213.929873  
       2025-07-25 23:00:00+00:00        322.0  213.949584  

[80 rows x 7 columns]

In [17]:
# get historical trades by symbol
req = StockTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 20,                                                # specify limit
)
stock_historical_data_client.get_stock_trades(req).df

exchange   price   size  id  \
symbol timestamp                                                      
AAPL   2025-07-21 08:00:00.033819+00:00        K  211.70   17.0   1   
       2025-07-21 08:00:00.036344+00:00        K  211.70   10.0   2   
       2025-07-21 08:00:00.036381+00:00        K  211.70   23.0   3   
       2025-07-21 08:00:00.036388+00:00        K  211.40   21.0   4   
       2025-07-21 08:00:00.037171+00:00        K  212.00    5.0   5   
       2025-07-21 08:00:00.040719+00:00        K  211.71    1.0   6   
       2025-07-21 08:00:00.040752+00:00        K  211.40    4.0   7   
       2025-07-21 08:00:00.040752+00:00        K  211.28    7.0   8   
       2025-07-21 08:00:00.040752+00:00        K  211.28   10.0   9   
       2025-07-21 08:00:00.040752+00:00        K  211.28    5.0  10   
       2025-07-21 08:00:00.040769+00:00        K  211.24    5.0  11   
       2025-07-21 08:00:00.040789+00:00        K  211.23    5.0  12   
       2025-07-21 08:00:00.040790+00:00        K  211.22  133.0  13   
       2025-07-21 08:00:00.040800+00:00        K  211.22  500.0  14   
       2025-07-21 08:00:00.041017+00:00        P  211.88   42.0   1   
       2025-07-21 08:00:00.041473+00:00        K  211.28    3.0  15   
       2025-07-21 08:00:00.041481+00:00        K  211.68    2.0  16   
       2025-07-21 08:00:00.042777+00:00        K  211.94    1.0  17   
       2025-07-21 08:00:00.043474+00:00        K  211.94    2.0  18   
       2025-07-21 08:00:00.043587+00:00        K  211.84    6.0  19   

                                        conditions tape  
symbol timestamp                                         
AAPL   2025-07-21 08:00:00.033819+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.036344+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.036381+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.036388+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.037171+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.040719+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.040752+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.040752+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.040752+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.040752+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.040769+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.040789+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.040790+00:00     [@, T]    C  
       2025-07-21 08:00:00.040800+00:00     [@, T]    C  
       2025-07-21 08:00:00.041017+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.041473+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.041481+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.042777+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.043474+00:00  [@, T, I]    C  
       2025-07-21 08:00:00.043587+00:00  [@, T, I]    C

In [19]:
# get historical quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                      # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 20,                                              # specify limit
)
stock_historical_data_client.get_stock_quotes(req).df

bid_price  bid_size bid_exchange  \
symbol timestamp                                                            
AAPL   2025-07-21 08:00:00.001107+00:00     211.20       4.0            Z   
       2025-07-21 08:00:00.014338+00:00     211.53       2.0            Q   
       2025-07-21 08:00:00.014427+00:00     211.53       2.0            Q   
       2025-07-21 08:00:00.043593+00:00     211.70       1.0            K   
       2025-07-21 08:00:00.043911+00:00     211.53       2.0            Q   
       2025-07-21 08:00:00.046886+00:00     211.88       1.0            P   
       2025-07-21 08:00:00.082530+00:00     211.88       1.0            P   
       2025-07-21 08:00:00.082542+00:00     211.88       1.0            P   
       2025-07-21 08:00:00.125305+00:00     211.75       5.0            P   
       2025-07-21 08:00:00.174328+00:00     211.75       5.0            P   
       2025-07-21 08:00:00.174341+00:00     211.75       5.0            P   
       2025-07-21 08:00:00.886456+00:00     211.75       4.0            P   
       2025-07-21 08:00:00.887040+00:00     211.50       1.0            P   
       2025-07-21 08:00:00.887099+00:00     211.50       1.0            P   
       2025-07-21 08:00:00.887337+00:00     211.50       1.0            P   
       2025-07-21 08:00:00.891034+00:00     211.50       1.0            P   
       2025-07-21 08:00:00.995769+00:00     211.62       2.0            Q   
       2025-07-21 08:00:01.005775+00:00     211.63       2.0            Q   
       2025-07-21 08:00:01.008449+00:00     211.50       1.0            P   
       2025-07-21 08:00:01.008464+00:00     211.53       2.0            Q   

                                         ask_price  ask_size ask_exchange  \
symbol timestamp                                                            
AAPL   2025-07-21 08:00:00.001107+00:00     212.19       3.0            Z   
       2025-07-21 08:00:00.014338+00:00     212.19       3.0            Z   
       2025-07-21 08:00:00.014427+00:00     211.92       4.0            Q   
       2025-07-21 08:00:00.043593+00:00     211.92       4.0            Q   
       2025-07-21 08:00:00.043911+00:00     211.92       4.0            Q   
       2025-07-21 08:00:00.046886+00:00     211.92       4.0            Q   
       2025-07-21 08:00:00.082530+00:00     212.00      18.0            K   
       2025-07-21 08:00:00.082542+00:00     211.94       4.0            Q   
       2025-07-21 08:00:00.125305+00:00     211.94       4.0            Q   
       2025-07-21 08:00:00.174328+00:00     212.00      18.0            K   
       2025-07-21 08:00:00.174341+00:00     211.95       4.0            Q   
       2025-07-21 08:00:00.886456+00:00     211.95       4.0            Q   
       2025-07-21 08:00:00.887040+00:00     211.95       4.0            Q   
       2025-07-21 08:00:00.887099+00:00     211.87       1.0            P   
       2025-07-21 08:00:00.887337+00:00     211.87       1.0            K   
       2025-07-21 08:00:00.891034+00:00     211.95       4.0            Q   
       2025-07-21 08:00:00.995769+00:00     211.95       4.0            Q   
       2025-07-21 08:00:01.005775+00:00     211.95       4.0            Q   
       2025-07-21 08:00:01.008449+00:00     211.95       4.0            Q   
       2025-07-21 08:00:01.008464+00:00     211.95       4.0            Q   

                                        conditions tape  
symbol timestamp                                         
AAPL   2025-07-21 08:00:00.001107+00:00        [R]    C  
       2025-07-21 08:00:00.014338+00:00        [R]    C  
       2025-07-21 08:00:00.014427+00:00        [R]    C  
       2025-07-21 08:00:00.043593+00:00        [R]    C  
       2025-07-21 08:00:00.043911+00:00        [R]    C  
       2025-07-21 08:00:00.046886+00:00        [R]    C  
       2025-07-21 08:00:00.082530+00:00        [R]    C  
       2025-07-21 08:00:00.082542+00:00        [R]    C  
       2025-07-21 08:00:00.125305+00:00        [R]    C  
       2025-0

In [20]:
corporate_actions_client = CorporateActionsClient(api_key, secret_key)
corporate_actions_client.get_corporate_actions(CorporateActionsRequest(
    start=datetime(2020, 1, 1),
    symbols=[symbol]
)).df

,id,symbol,cusip,rate,special,foreign,process_date,ex_date,record_date,payable_date,new_rate,old_rate,due_bill_redemption_date
corporate_action_type,,,,,,,,,,,,,
cash_dividends,23407e01-4eef-4c1e-a2dd-f585c7eabcd0,AAPL,,0.770,False,False,2020-02-07,2020-02-07,None,None,NaN,NaN,NaN
cash_dividends,235946d4-2605-4e5c-9699-9bfd00452ec0,AAPL,037833100,0.820,False,False,2020-05-14,2020-05-08,2020-05-11,2020-05-14,NaN,NaN,NaN
cash_dividends,d09386ae-0c2b-4280-8aa0-295e545354b1,AAPL,037833100,0.820,False,False,2020-08-13,2020-08-07,2020-08-10,2020-08-13,NaN,NaN,NaN
cash_dividends,43032b74-6743-4fef-9a10-de51c464a480,AAPL,037833100,0.205,False,False,2020-11-12,2020-11-06,2020-11-09,2020-11-12,NaN,NaN,NaN
cash_dividends,65570a16-85fa-4c01-b448-381a789c9248,AAPL,037833100,0.205,False,False,2021-02-11,2021-02-05,2021-02-08,2021-02-11,NaN,NaN,NaN
cash_dividends,6a0daa4a-c7a1-47f6-8d0d-090a469abb9d,AAPL,037833100,0.220,False,False,2021-05-13,2021-05-07,2021-05-10,2021-05-13,NaN,NaN,NaN
cash_dividends,141b5396-727a-417f-8983-581281c6ae67,AAPL,037833100,0.220,False,False,2021-08-12,2021-08-06,2021-08-09,2021-08-12,NaN,NaN,NaN
cash_dividends,9088cb26-23c0-4fd0-ab86-dd7622ee610e,AAPL,037833100,0.220,False,False,2021-11-11,2021-11-05,2021-11-08,2021-11-11,NaN,NaN,NaN
cash_dividends,9ba01618-fa72-4d1d-8496-2e3b3e64c082,AAPL,037833100,0.220,False,False,2022-02-10,2022-02-04,2022-02-07,2022-02-10,NaN,NaN,NaN


In [27]:
import pandas as pd

def get_data(symbol, start_date=datetime.now(ZoneInfo("America/New_York"))-timedelta(days=5), end_date=datetime.now(ZoneInfo("America/New_York")), timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour)):
    # check if the data is already in the csv file
    if os.path.exists(f"data/{symbol}_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}_{timeframe.amount}_{timeframe.unit}.csv"):
        return pd.read_csv(f"data/{symbol}_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}_{timeframe.amount}_{timeframe.unit}.csv")
    
    # get historical bars by symbol
    # ref. https://docs.alpaca.markets/reference/stockbars-1
    req = StockBarsRequest(
        symbol_or_symbols = symbol,
        timeframe=timeframe, # specify timeframe
        start = start_date,                          # specify start datetime, default=the beginning of the current day.
        end_date=end_date,                                        # specify end datetime, default=now
        # limit = 2,                                               # specify limit
    )
    data = stock_historical_data_client.get_stock_bars(req).df

    # save the data to a csv file with start date and symbol
    data.to_csv(f"data/{symbol}_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}_{timeframe.amount}_{timeframe.unit}.csv")
    return data

data = get_data("TSLA")
data.head()

open    high     low   close     volume  \
symbol timestamp                                                              
TSLA   2025-07-21 08:00:00+00:00  333.39  335.00  332.88  334.40   343595.0   
       2025-07-21 09:00:00+00:00  334.36  334.97  334.22  334.32   134755.0   
       2025-07-21 10:00:00+00:00  334.37  334.75  333.94  334.13   201851.0   
       2025-07-21 11:00:00+00:00  334.11  335.20  333.44  335.13   389300.0   
       2025-07-21 12:00:00+00:00  333.67  335.61  333.42  334.41  1217417.0   

                                  trade_count        vwap  
symbol timestamp                                           
TSLA   2025-07-21 08:00:00+00:00      12930.0  334.400432  
       2025-07-21 09:00:00+00:00       5281.0  334.535240  
       2025-07-21 10:00:00+00:00       6607.0  334.319131  
       2025-07-21 11:00:00+00:00      12063.0  334.544178  
       2025-07-21 12:00:00+00:00      27033.0  334.683091